# Generate and edit the AudioSet ontology

Using [Owlready2](https://owlready2.readthedocs.io/en/latest/index.html) package

First import the library and define the local `ontologies` folder. If an URL is given, first searches for a local copy of the OWL file and, if not found, tries to download it from the Internet.

In [1]:
from owlready2 import *

onto_path.append("/notebooks/ontologies")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## Create and load ontologies

Create the AudioSet ontology and load the Pizza ontology from the Internet (for example purpose)

In [2]:
audioset_onto = get_ontology("https://w3id.org/audioset/ontology")

pizza_onto = get_ontology("http://www.lesfleursdunormal.fr/static/_downloads/pizza_onto.owl").load()

## Add classes and properties

See [Owlready2 documentation](https://owlready2.readthedocs.io/en/latest/index.html) for:
* [Properties](https://owlready2.readthedocs.io/en/latest/properties.html)
* [Classes](https://owlready2.readthedocs.io/en/latest/class.html)

In [6]:
with audioset_onto:
    class HumanSounds(Thing):
        pass
    class HumanVoice(HumanSounds):
        pass
    class Accent(Thing):
        pass
    class has_accent(ObjectProperty):
        domain    = [HumanVoice]
        range     = [Accent]
    class description(ObjectProperty):
        range     = [str]

## Explore an ontology

List an ontology classes and properties

In [7]:
print(audioset_onto.HumanVoice.iri)
print(list(audioset_onto.classes()))
print(list(audioset_onto.object_properties()))

https://w3id.org/audioset/ontology#HumanVoice
[ontology.HumanSounds, ontology.HumanVoice, ontology.Accent]
[ontology.has_accent, ontology.description]


## Add metadata to the ontology

In [8]:
audioset_onto.metadata.comment.append("my comment")

## Save ontology file

In [9]:
# Formats available: rdfxml or ntriples
audioset_onto.save(file = "ontologies/audioset.owl", format = "rdfxml")